In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [47]:
class RootMeanSquarredError(keras.metrics.Metric):
    def __init__(self, name = "rmse", **kwargs ):
        super().__init__(name = name ,**kwargs)
        
        self.mse_sum = self.add_weight(name = "mse_sum", initializer = 'zeros')
        self.total_samples = self.add_weight(name='total_samples', initializer ='zeros', dtype="int32")
        
    def update_state(self, y_true, y_pred, sample_weight = None):
        y_true = tf.one_hot(y_true, depth = tf.shape(y_pred) [1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign(num_samples)
        
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))
    
    
    def reset_state(self):
        self.mse_sum.assign(0,)
        self.total_samples.assign(0)

In [48]:
RMSE = RootMeanSquarredError()

In [49]:
def get_model():
    inputs = keras.Input(shape=(28*28))
    features = keras.layers.Dense(512, activation='relu')(inputs)
    drop = keras.layers.Dropout(0.5)(features)
    outputs = keras.layers.Dense(10, activation='softmax')(drop)
    
    model = keras.Model(inputs = inputs , outputs = outputs)
    
    return model

In [55]:
model = get_model()

In [56]:
model.compile(optimizer = 'rmsprop',
             loss= 'sparse_categorical_crossentropy',
             metrics = ['accuracy', RMSE])

In [57]:
from keras.datasets import mnist
(train_images , train_labels ) , (test_images , test_labels) = mnist.load_data()

In [58]:
train_images = train_images.reshape((60000, 28*28)).astype('float32') / 255
test_images  = test_images.reshape((10000, 28*28)).astype('float32') /255

In [59]:
model.fit(train_images , train_labels ,epochs = 10 , validation_data =( test_images , test_labels))

Epoch 1/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2803 - accuracy: 0.9171 - rmse: 7.4437 - val_loss: 0.1279 - val_accuracy: 0.9617 - val_rmse: 5.3088
Epoch 2/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1601 - accuracy: 0.9556 - rmse: 7.3669 - val_loss: 0.1186 - val_accuracy: 0.9677 - val_rmse: 5.3333
Epoch 3/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1372 - accuracy: 0.9638 - rmse: 7.3500 - val_loss: 0.1031 - val_accuracy: 0.9748 - val_rmse: 5.3359
Epoch 4/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1291 - accuracy: 0.9677 - rmse: 7.4964 - val_loss: 0.1023 - val_accuracy: 0.9751 - val_rmse: 5.3358
Epoch 5/10
1875/1875 [==============================] - 20s 10ms/step - loss: 0.1172 - accuracy: 0.9710 - rmse: 7.4430 - val_loss: 0.1015 - val_accuracy: 0.9763 - val_rmse: 5.3378
Epoch 6/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1124 - accuracy: 0.9734 